In [1]:
from music21 import converter, instrument, note, chord
import sys
import numpy as np
from imageio import imwrite
from glob import glob

In [2]:
def extractNote(element):
    return int(element.pitch.ps)

def extractDuration(element):
    return element.duration.quarterLength

In [3]:
def get_notes(notes_to_parse):

    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    durations = []
    notes = []
    start = []

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            if element.isRest:
                continue

            start.append(element.offset)
            notes.append(extractNote(element))
            durations.append(extractDuration(element))
                
        elif isinstance(element, chord.Chord):
            if element.isRest:
                continue
            for chord_note in element:
                start.append(element.offset)
                durations.append(extractDuration(element))
                notes.append(extractNote(chord_note))

    return {"start":start, "pitch":notes, "dur":durations}


In [7]:
def midi2image(midi_path, max_repetitions = float("inf"), resolution = 0.25, lowerBoundNote = 21, upperBoundNote = 127, maxSongLength = 300):
    mid = converter.parse(midi_path)

    instruments = instrument.partitionByInstrument(mid)

    data = {}

    try:
        i=0
        for instrument_i in instruments.parts:
            notes_to_parse = instrument_i.recurse()

            notes_data = get_notes(notes_to_parse)
            if len(notes_data["start"]) == 0:
                continue

            if instrument_i.partName is None:
                data["instrument_{}".format(i)] = notes_data
                i+=1
            else:
                data[instrument_i.partName] = notes_data

    except:
        notes_to_parse = mid.flat.notes
        data["instrument_0"] = get_notes(notes_to_parse)

    for instrument_name, values in data.items():
        # https://en.wikipedia.org/wiki/Scientific_pitch_notation#Similar_systems

        pitches = values["pitch"]
        durs = values["dur"]
        starts = values["start"]

        index = 0
        while index < max_repetitions:
            matrix = np.zeros((upperBoundNote-lowerBoundNote,maxSongLength))


            for dur, start, pitch in zip(durs, starts, pitches):
                dur = int(dur/resolution)
                start = int(start/resolution)

                if not start > index*(maxSongLength+1) or not dur+start < index*maxSongLength:
                    for j in range(start,start+dur):
                        if j - index*maxSongLength >= 0 and j - index*maxSongLength < maxSongLength:
                            matrix[pitch-lowerBoundNote,j - index*maxSongLength] = 255

            if matrix.any(): # If matrix contains no notes (only zeros) don't save it
                imwrite(os.getcwd() + "/images/" + midi_path.split("/")[-1].replace(".midi",f"_{instrument_name}_{index}.png"),matrix.astype(np.uint8))
                index += 1
            else:
                break


In [20]:
a = os.getcwd() + "/maestro/2009/"
filenames = os.listdir(a)

In [21]:
for i in range(len(filenames)):
    midi2image(os.getcwd() + "/maestro/2009/" + filenames[i])

In [22]:
a = os.getcwd() + "/maestro/2011/"
filenames = os.listdir(a)
for i in range(len(filenames)):
    midi2image(os.getcwd() + "/maestro/2011/" + filenames[i])

In [23]:
a = os.getcwd() + "/maestro/2013/"
filenames = os.listdir(a)
for i in range(len(filenames)):
    midi2image(os.getcwd() + "/maestro/2013/" + filenames[i])

In [ ]:
a = os.getcwd() + "/maestro/2014/"
filenames = os.listdir(a)
for i in range(len(filenames)):
    midi2image(os.getcwd() + "/maestro/2014/" + filenames[i])

In [25]:
a = os.getcwd() + "/maestro/2015/"
filenames = os.listdir(a)
for i in range(len(filenames)):
    midi2image(os.getcwd() + "/maestro/2015/" + filenames[i])

In [26]:
a = os.getcwd() + "/maestro/2017/"
filenames = os.listdir(a)
for i in range(len(filenames)):
    midi2image(os.getcwd() + "/maestro/2017/" + filenames[i])

In [ ]:
a = os.getcwd() + "/maestro/2018/"
filenames = os.listdir(a)
for i in range(len(filenames)):
    midi2image(os.getcwd() + "/maestro/2018/" + filenames[i])